Exercise 8: Classifying Supersymmetry
=========================

The [Large Hadron Collider] (LHC) is a machine for smashing high-speed atomic nuclei into
each other.  Each collision is observed by one of the two large detectors (ATLAS and CMS), which are
like huge digital cameras taking snapshots of the particles generated in the
collision. The LHC generates 40,000,000 collision events per second per experiment, and a CMS "photograph" of
one such event is around one MB in size.  Storing every event, let alone analyzing it,
is completely infeasible.

Fortunately, in a hadron collider, as in presentations of vacation pictures, the vast majority
of photographs are "boring", which means no new physics is expected to be observed there.
We will classify those events as **background** and assign to it the label `0`.  The collision events
interesting for our application are classified as **signal** and have the label `1`.  Note that an
event being signal does not mean it contains new physics; it just means it is a collision
where new physics *may* be seen.

One theory for which the LHC is on the lookout is [supersymmetry], or SUSY for short.  We won't 
have time to go into the details: suffice it to say, it is a much-sought-after theory by high-energy 
physicists because if true, it would solve a series of problems with the standard model and greatly
simplify the maths.  However, until now nature has refused to play along and no experimental evidence
of SUSY has been found.

In this exercise you will have to train a model which classifies events as signal or background for
the purposes of SUSY, given 18 hand-picked `variables` observed in each event (the explanations
are given in `variable_names`).  The [dataset] has 5,000,000 events, generated from simulations, together
with their true classification (`labels`).

[Large Hadron Collider]: https://en.wikipedia.org/wiki/Large_Hadron_Collider
[supersymmetry]: https://en.wikipedia.org/wiki/Supersymmetry
[dataset]: https://www.nature.com/articles/ncomms5308

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as pl

In [ ]:
# Load the dataset from a binary file
with np.load(os.path.expanduser("~/shared/susy.npz")) as _datafile:
    labels = _datafile["labels"].astype(int)
    variables = _datafile["variables"].astype(float)
    variable_names = list(_datafile["variable_names"].astype(str))

In [ ]:
variable_names

In [ ]:
variables.shape

In [ ]:
labels.shape

In [ ]:
labels[:10]

Step 1: Partition the data into a training and a test set
---------------------------------------------------------------------------

Fill variables `X_train`, `y_train` with design matrix and labels
of the training set, and the same for `X_test` and `y_test` for the
test set.

You should put 90% of observations into the training set and 10% into the
test set.

The data has been randomized, so you do not need to do this here
(but can if you like.)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert (X_train, y_train, X_test, y_test) is not None
assert X_train.shape[:1] == y_train.shape
assert X_test.shape[:1] == y_test.shape
np.testing.assert_allclose(y_train.size / y_test.size, 9)

Step 2: Train a logistic model
--------------------------------------

Our days of writing all of those fitting things ourselves are now behind us.
We are going to leverage the power of the scikit-learn (`sklearn`) package.
This also means I will step back a little bit and you will have to figure
out more things on your own.

Let us use a `sklearn.linear_model.SGDClassifier` (see the [documentation]),
which is a binary classifier using stochastic gradient descent.

Training with sklearn is two-step procedure:

 1. first, you create a `model`, which encodes both which predictor/classifier
    we are using and how we are going to train it. In our case this is a 
    `SGDClassifier`.   As parameters you should make sure to use the loss function
    for logistic regression and use early stopping.
    
 2. second, we are going to perform the regression on the data set using
    the `fit` method of the model.  Here you should pass the training data.

[documentation]: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier

In [ ]:
import sklearn.linear_model

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert (model,) is not None
assert model.__class__.__name__ == "SGDClassifier"
assert model.score(X_train, y_train) < 1

Step 3: Predict values on the test set and check errors
---------------------------------------------

Now that the model is trained, we can use it to predict values for
the test set, which you should store in `yhat_test`.

Afterwards, compute the in- and out-of-sample error and store it
in `E_in` and `E_out`, respectively.

**Hint**: For both tasks you may have to rummage through the methods of `model` a
little bit or read the documentation.  The error is called "score"
in sklearn terminology.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print ("in-sample (training) error:%10.5f" % E_in)
print ("out-of-sample (test) error:%10.5f" % E_out)

In [ ]:
assert (yhat_test, E_in, E_out) is not None
_am_right = (yhat_test == y_test).sum()
assert _am_right > 350_000, "Poor prediction accuracy" 
assert _am_right < 499_000, "Too good prediction accuracy"
assert 0 < E_in < 1
assert 0 < E_out < 1

Step 4: Evaluate prediction quality
------------------------------

Compute and print/plot the **confusion matrix** (on the test set).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

You should also compute the accuracy, sensitivity, and specificity
and store it in `accuracy`, `sensitivity`, and `specificity`,
respectively and then print those values as well.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert 0.5 < accuracy < 0.95
assert 0.5 < sensitivity < 0.95
assert 0.5 < specificity < 0.95
assert sensitivity < specificity


Draw conclusions based on your metrics:

 1. Is the dataset significantly imbalanced?
 
 2. Observe that the sensitivity is smaller than the
    specificity.  What does this mean? 
  
 3. Is this model suitable for finding regions of interest
    for SUSY search in the data? Why or why not?

YOUR ANSWER HERE

Step 5: Debugging the model
------------------------------

In an effort to improve the model, one question one can naturally ask is **which**
features are modelled accurately and which features are not.

For this, put all observations of the test design matrix `X_test` which were
predicted incorrectly into `X_fail`, and those which were predicted correctly
into `X_success`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert X_fail.shape[1] == X_test.shape[1]
assert X_fail.shape[0] == (yhat_test != y_test).sum()
assert X_success.shape[1] == X_test.shape[1]
assert X_success.shape[0] == (yhat_test == y_test).sum()


Now let us do a (quite crude!) analysis of where this may go wrong:
If values for feature, say $x_k$, are on average very different in the
training set than in the test set, then we can probably assume that the
dependency on this feature cannot be properly captured by our model.

To study this:

 1. let us first take the **mean** over hits and misses for each feature (mean over 
    the columns of `X_fail` and `X_success`) - let us call this $\bar x^\mathrm{hit}_k$
    and $\bar x^\mathrm{miss}_k$, respectively.
    
 2. to make sure that our "distances" are meaningful, let us also compute
    the **standard deviation** over all observations for each feature (standard
    deviation over the columns of `variables`) - we call this $\sigma_k$.

(Hint: you can use the `mean` and `std` function together with the axis
argument.)

Now plot the $\bar x^\mathrm{hit}_k / \sigma_k$ and $\bar x^\mathrm{miss}_k / \sigma_k$
over $k$. To spot the difference, I suggest plotting these as two lines in 
the same plot.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

 1. According to the above plot, which features are most likely involved in poor
    predictions?

 2. How could we modify the design matrix such that we allow the logistic model
    to treat these features (potentially) better?

YOUR ANSWER HERE